# Modelling 

### Categorical Encoding

In [ ]:
# One-hot encoding
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Indexing the categorical columns
vendor_indexer = StringIndexer(inputCol="VendorID", outputCol="VendorID_index")
ratecode_indexer = StringIndexer(inputCol="RatecodeID", outputCol="RatecodeID_index")
payment_type_indexer = StringIndexer(inputCol="payment_type", outputCol="payment_type_index")

# OneHotEncoding the indexed columns
vendor_encoder = OneHotEncoder(inputCol="VendorID_index", outputCol="VendorID_vec")
ratecode_encoder = OneHotEncoder(inputCol="RatecodeID_index", outputCol="RatecodeID_vec")
payment_type_encoder = OneHotEncoder(inputCol="payment_type_index", outputCol="payment_type_vec")

# Creating a pipeline to chain indexers and encoders
pipeline = Pipeline(stages=[vendor_indexer, ratecode_indexer, payment_type_indexer,
                            vendor_encoder, ratecode_encoder, payment_type_encoder])

# Fit the pipeline and transform the data
model = pipeline.fit(combined)
encoded_df = model.transform(combined)

# Drop the original columns after encoding
encoded_df = encoded_df.drop("VendorID", "RatecodeID", "payment_type")


## Data split

## Linear regression

## Random Forest 